# Защита персональных данных клиентов

# Описание проекта

Необходимо защитить данные клиентов страховой компании «Хоть потоп». 

Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Обосновать корректность его работы.

Необходимо защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Инструкция по выполнению проекта**

1. Загрузить и изучить данные.

2. Ответить на вопрос и обосновать решение. Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
 
   - a. Изменится. Приведите примеры матриц.
   
   - b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

3. Предложить алгоритм преобразования данных для решения задачи. Обосновать, почему качество линейной регрессии не поменяется.

4. Запрограммировать этот алгоритм, применив матричные операции. Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применить метрику R2.

**Описание данных**

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

**Импорт необходимых библиотек**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from colorama import Fore, Style # модуль пришлось доинсталлировать в тренажере

import warnings

warnings.filterwarnings('ignore')

RANDOM = 1234

**Загрузка датасета**

In [2]:
try:
    data = pd.read_csv("/datasets/insurance.csv")
except FileNotFoundError:
    data = pd.read_csv(
        "https://code.s3.yandex.net/datasets/insurance.csv")

**Первые пять строк датасета**

In [3]:
print(f"{Fore.GREEN}{Style.BRIGHT}Первые пять строк датасета:{Style.RESET_ALL}")
display(data.head(5))

Первые пять строк датасета:


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


**Проверка на пропущенные значения**

In [4]:
print(f"{Fore.RED}{Style.BRIGHT}Пропуски в данных:{Style.RESET_ALL}")
display(data.isna().sum())

Пропуски в данных:


Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

**Информация по датафрейму**

In [5]:
print(f"{Fore.GREEN}{Style.BRIGHT}Информация по датафрейму:{Style.RESET_ALL} \n") 
data.info()

Информация по датафрейму: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Т.к. `Зарплата` и `Возраст` у нас целочисленные, можно помеять тип данных на более оптимальный. В колонках `Пол`, `Члены семьи` и `Страховые выплаты` значения только `0` и `1`. Тут тоже пожно помеять на более оптимальный тип данных.

In [6]:
data[['Пол', 'Возраст', 'Члены семьи', 'Страховые выплаты'
      ]] = data[['Пол', 'Возраст', 'Члены семьи',
                 'Страховые выплаты']].astype('int8')
data['Зарплата'] = data['Зарплата'].astype('int64')

**Проверка на дубликаты**

In [7]:
print(f"{Fore.RED}{Style.BRIGHT}Дубликаты в данных:{Style.RESET_ALL}")
display(data.duplicated().sum())

Дубликаты в данных:


153

In [8]:
display(
    data.loc[data.duplicated(keep=False)].sort_values(by='Зарплата').head(20))

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2955,1,32,21600,0,0
2988,1,32,21600,0,0
361,0,50,24700,1,2
2869,0,50,24700,1,2
333,0,32,25600,1,0
4230,0,32,25600,1,0
1378,0,36,26400,0,0
2723,0,36,26400,0,0
1002,1,34,26900,0,0
1140,1,34,26900,0,0


В данных присутствуют дубликаты, но т.к. стотит другая задача исседования оставим как есть.

**Данные подготовлены к дальнейшему исследованию.**

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**До проведения преобразований рассчитаем метрику `R2` для линейной регрессии по вышеуказанной формуле.**

**Выделение признаков и целевого признака.**

In [9]:
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

**Разделение на тренировочную и тестовую выборки.**

In [10]:
train_features, test_features, train_target, test_target = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM)

**Модель по исходным формулам для предсказания значений.**

In [11]:
class LinRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones(
            (train_features.shape[0], 1)), train_features),
                           axis=1)
        y = train_target
        w = (np.linalg.inv(X.T.dot(X)).dot(X.T)).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [12]:
model = LinRegression()
model.fit(train_features, train_target)
predict = model.predict(test_features)

score_r2 = round(r2_score(test_target, predict), 6)

In [13]:
print(
    f"{Fore.RED}{Style.BRIGHT}Метрика R2 для самостоятельно построенной модели: {Fore.BLUE}{Style.BRIGHT}{score_r2}{Style.RESET_ALL}"
)

Метрика R2 для самостоятельно построенной модели: 0.424962


**Для сравнения правильности построения модели и результатов её работы найдем метрику при помощи линейной регрессии из `sklearn`.**

In [14]:
model_skl = LinearRegression()
model_skl.fit(train_features, train_target)
predict_skl = model_skl.predict(test_features)

score_r2_skl = round(r2_score(test_target, predict_skl), 6)

In [15]:
print(
    f"{Fore.RED}{Style.BRIGHT}Метрика R2 для модели из sklearn: {Fore.GREEN}{Style.BRIGHT}{score_r2_skl}{Style.RESET_ALL}"
)

Метрика R2 для модели из sklearn: 0.424962


Модели построены. 

**Вопрос исследования: Изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу** 

**Ответ: не изменится** 

**Обоснование:** 

Необходимо доказать, что при умножении исходной матрицы $X$ на обратимую матрицу $P$ предсказания модели и соответственно метрика `R2` не изменятся.

Обозначим новую матрицу $Q$ как произведение матриц $X$ и $P$
$$Q = XP$$

Формула обучения теперь выглядит так:    $w1 = (Q^T Q)^{-1} Q^{T} y$

Заменим в формуле $Q$ на $X$ и $P$: $w1 = ((XP)^T (XP))^{-1} (XP)^{T} y$

Предсказания будут выглядеть соответсвенно: $a1 = (XP)((XP)^T (XP))^{-1} (XP)^{T} y$

Для преобразования и раскрытия скобок необходимо учитывать свойства матриц:

 - $(AB)^{-1} = B^{-1}A^{-1}$
 - $(AB)^{T} = B^{T}A^{T}$
 - $AA^{-1} = E$ , где $E$ - единичная матрица 
 - $AE = EA = A$
 
Преобразуем $w1 = ((XP)^T (XP))^{-1} (XP)^T y$ 

$$w1 = ((XP)^T (XP))^{-1} (XP)^T y = (P^{T}X^{T}(XP))^{-1} P^{T}X^{T}y = $$
$$ = (P^{T}(X^{T}X)P)^{-1} P^{T}X^{T} y = $$
$$ = P^{-1}(X^{T}X)^{-1}(P^{T})^{-1} P^{T}X^{T} y = $$
согласно свойствам $ (P^{T})^{-1} P^{T} = E$

$$w1 = P^{-1}(X^{T}X)^{-1} EX^{T} y = P^{-1}(X^{T}X)^{-1} X^{T} y$$


Из формулы видно, что $(X^{T}X)^{-1}X^{T}y$ это есть $w = (X^T X)^{-1} X^T y$ из условия задачи.

Соотвественно получаем: $w1 = P^{-1} w$

Подставим в это значение в исходную формулу: $a = Xw$

$a1 = Qw1 = (XP) P^{-1} w = X(P P^{-1})w = XEw = Xw$

В результате $a = a1$

**Что и требовалось доказать.**

## Алгоритм преобразования

**Алгоритм**

Для защиты данных нужно умножать матрицу признаков на обратимую матрицу, которая будет генерироваться случайно функцией `np.random.normal` и рассчитать метрику для линейной регрессии с новыми признаками.

**Обоснование**

В пункте 2 было доказано, что умножение признаков на обратимую матрицу не должно привести к изменениям результатов. Метрики для моделей с неизмененными признаками рассчитаны выше.  


Создаем рандомную матрицу:

In [16]:
n = features.shape[1]
random_matrix = np.random.normal(5, size=(n, n))

print(
    f"{Fore.LIGHTMAGENTA_EX}{Style.BRIGHT}Рандомная матрица 4х4:{Style.BRIGHT}{Style.RESET_ALL}\n{random_matrix}"
)

Рандомная матрица 4х4:
[[4.78170258 4.74340139 4.58569292 5.42453251]
 [5.027623   4.05058948 6.54070765 3.6272019 ]
 [4.24622491 7.13369984 4.0377412  3.49254707]
 [4.44292983 2.8160619  4.69872087 5.32668571]]


**Проверка на обратимость**

In [17]:
print(
    f"{Fore.RED}{Style.BRIGHT}Обратная матрица к рандомной матрице:{Style.BRIGHT}{Style.RESET_ALL}\n{np.linalg.inv(random_matrix)}"
)

Обратная матрица к рандомной матрице:
[[ 19.51138061   3.55752544  -8.35662745 -16.81309383]
 [ -3.16960915  -0.63916356   1.57136845   2.63277123]
 [ -9.66897884  -1.42455148   4.0046582    8.19090349]
 [ -6.0694489   -1.37277519   2.60688695   5.59420657]]


При умножении рандомной матрицы на обратную должна получиться единчная матрица.

In [18]:
print(
    f"{Fore.GREEN}{Style.BRIGHT}Единичная матрица:{Style.BRIGHT}{Style.RESET_ALL}\n{np.round(random_matrix @ np.linalg.inv(random_matrix))}"
)

Единичная матрица:
[[ 1. -0. -0.  0.]
 [ 0.  1.  0.  0.]
 [ 0. -0.  1.  0.]
 [ 0.  0. -0.  1.]]


**Матрица для проверки алгоритма готова**

## Проверка алгоритма

**Умножаем признаки на трейне и тесте на рандомную матрицу**

In [19]:
train_features_new = train_features.dot(random_matrix)
test_features_new = test_features.dot(random_matrix)

**Проверка результатов умножения**

In [20]:
train_features_new.columns = train_features.columns
train_features_new

,Пол,Возраст,Зарплата,Члены семьи
3958,204847.035295,343987.142682,194847.110193,168478.244038
4768,237109.704704,398196.936522,225526.500863,195009.151102
326,154374.016748,259144.374667,146875.463694,126951.734622
2026,189162.895306,317616.409809,179945.697263,155568.857421
586,216706.884010,363938.433222,206115.383285,178233.913151
...,...,...,...,...
664,150841.290995,253328.048880,143468.671630,124059.762511
3276,151324.490883,254086.322355,143945.408281,124457.642093
1318,171702.173462,288324.572708,163323.822320,141214.744033
723,175629.940661,294831.199469,167096.986245,144432.508156


In [21]:
test_features_new.columns = test_features.columns
test_features_new

,Пол,Возраст,Зарплата,Члены семьи
2706,149242.768796,250554.346207,141982.547317,122736.987203
2436,169998.409953,285467.734995,161700.230119,139815.895332
1201,150406.859178,252605.884905,143053.771110,123701.456069
1486,136993.490951,229918.039810,130356.285059,112657.453795
4286,116911.373893,196288.927546,111219.180743,96148.305684
...,...,...,...,...
3871,223970.376198,376100.250914,213038.522717,184203.790148
1075,212446.406840,356793.123330,202061.816967,174726.987657
40,170914.183421,286949.601681,162596.491531,140558.159404
1619,186134.064627,312575.794382,177043.646667,153087.574215


**Модель для предсказания значений и метрика `R2` c измененными признаками**

In [22]:
model_new = LinRegression()
model_new.fit(train_features_new, train_target)
predict_new = model_new.predict(test_features_new)

score_r2_new = round(r2_score(test_target, predict_new), 6)

In [23]:
print(
    f"{Fore.RED}{Style.BRIGHT}Метрика R2 для самостоятельно построенной модели с изменёнными признаками: {Fore.BLUE}{Style.BRIGHT}{score_r2_new}{Style.RESET_ALL}"
)

Метрика R2 для самостоятельно построенной модели с изменёнными признаками: 0.42494


**Модель и метрика `R2` для линейной регрессии из `sklearn`с изменёнными признаками.**

In [24]:
model_skl_new = LinearRegression()
model_skl_new.fit(train_features_new, train_target)
predict_skl_new = model_skl_new.predict(test_features_new)

score_r2_skl_new = round(r2_score(test_target, predict_skl_new), 6)

In [25]:
print(
    f"{Fore.RED}{Style.BRIGHT}Метрика R2 для модели из sklearn с изменёнными признаками: {Fore.GREEN}{Style.BRIGHT}{score_r2_skl_new}{Style.RESET_ALL}"
)

Метрика R2 для модели из sklearn с изменёнными признаками: 0.424962


**Результы сведённые в единую таблицу**

In [26]:
data_result = pd.DataFrame({
    'Модель': [
        'Собственная модель', 'Модель из sklearn',
        'Собственная модель с изменёнными признаками',
        'Модель из sklearn с изменёнными признаками'
    ],
    'Значение метрики R2':
    [score_r2, score_r2_skl, score_r2_new, score_r2_skl_new],
})

In [27]:
def color_dataframe(df):
    n = data_result.shape[1]
    if 'Собственная' in df["Модель"]:
        return ['background-color: yellow'] * n
    else:
        return ['background-color: cyan'] * n

In [28]:
data_result.style.apply(color_dataframe, axis=1)

,Модель,Значение метрики R2
0,Собственная модель,0.424962
1,Модель из sklearn,0.424962
2,Собственная модель с изменёнными признаками,0.424940
3,Модель из sklearn с изменёнными признаками,0.424962


**Все показатели без изменений**

## Вывод

**В результате исследования было доказано что можно зашифровать данные клиентов при умножении исходной матрицы признаков на обратимую матрицу.**

**Качество линейной регресии не изменилось от использования исxодных данных и данных, умноженных на обратимую матрицу.**